Recommender system
This reminds me of; https://en.wikipedia.org/wiki/Netflix_Prize

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
from scipy import optimize as op

In [2]:
data = sio.loadmat('./data/ex8_movies.mat')
data_params = sio.loadmat('./data/ex8_movieParams.mat')

The dataset has nu = 943 users, and nm = 1682 movies

In [3]:
result = {}
with open('./data/movie_ids.txt',encoding="latin-1") as f:
    for line in f:
        result[int(line.split()[0])] = line

In [4]:
Y = data['Y'] # rating matrix; Movies x Users (1 to 5)
R = data['R'] # indicator matrix; Movies x Users
print(Y.shape)

(1682, 943)


In [5]:
# best rated with at least 100 ratings
idx = np.where(R.sum(axis=1) > 100)
avg = Y[idx].sum(axis=1) / R[idx].sum(axis=1)
best = np.argmax(avg)
print('best rated: ', result[best])
print(best, R[best].sum(), avg[best])

best rated:  231 Batman Returns (1992)

231 101 4.491071428571429


In [6]:
X = data_params['X']
print(X.shape)
theta = data_params['Theta']
print(theta.shape)

(1682, 10)
(943, 10)


In [7]:
rolled_parameters = np.append(X, theta)

In [8]:
def unroled_parameters(rolled_parameters, number_movies, number_users, num_features):
    X = rolled_parameters[0:(number_movies*num_features)].reshape((number_movies, num_features))
    theta = rolled_parameters[(number_movies*num_features):].reshape((number_users, num_features))
    return X, theta

In [9]:
def cost_function(rolled_parameters, Y, R, num_features, _lambda=1.5):
    
    # unroll and reshape
    number_movies, number_users = Y.shape
    X, theta = unroled_parameters(rolled_parameters, number_movies, number_users, num_features) 

    cost_to_sum = ((X @ theta.T ) - Y)**2
    cost = 1/2 * (cost_to_sum * R).sum()
    print(cost)
    reg = (_lambda / 2) * ((theta ** 2).sum() + (X ** 2).sum())
    
    return cost + reg

In [10]:
cost_function(rolled_parameters, Y, R, 10)

27918.64012454421


34821.703613072226

In [11]:
# Reduce data set to run faster
num_users = 4
num_movies = 5
num_features = 3
X_red = X[:num_movies, :num_features]
theta_red = theta[:num_users, :num_features]
Y_red = Y[:num_movies, :num_users]
R_red = R[:num_movies, :num_users]
rolled_parameters_red = np.append(X_red, theta_red)

In [12]:
cost_function(rolled_parameters_red, Y_red, R_red, num_features)

22.224603725685675


31.34405624427422

In [13]:
def gradient(rolled_parameters, Y, R, num_features, _lambda=1.5):
    
    # unroll and reshape
    number_movies, number_users = Y.shape
    X, theta = unroled_parameters(rolled_parameters, number_movies, number_users, num_features) 
    
    # loop over movies to compute gradient X
    X_grad = np.zeros((number_movies, num_features))
    for i in range(number_movies):
        idx = np.where( R[i,:] == 1)[0]
        theta_tmp = theta[idx,:]
        Y_tmp = Y[i,idx].reshape((len(Y[i,idx]),1))
        
        x = X[i,:].reshape((len(X[i,:]),1))
        
        X_grad[i,:] = (((theta_tmp @ x) - Y_tmp) * theta_tmp).sum(axis=0) + (_lambda * x).reshape(num_features)
        
    # loop over users to compute gradient theta
    theta_grad = np.zeros((number_users, num_features))
    for j in range(num_users):
        idx = np.where( R[:,j] == 1)[0]
        X_tmp = X[idx,:]
        Y_tmp = Y[idx,j].reshape((len(Y[idx,j]),1))
        
        thetaj = theta[j,:].reshape((len(theta[j,:]),1))
        
        theta_grad[j,:] = (((X_tmp @ thetaj) - Y_tmp) * X_tmp).sum(axis=0) + (_lambda * thetaj).reshape(num_features)
        
    # concat
    return np.append(X_grad, theta_grad)


In [14]:
# gradient checking

In [15]:
def check_gradient(rolled_parameters_red, Y_red, R_red, num_features):
    e = 1e-4
    n = rolled_parameters_red.shape[0]
    
    to_check = np.zeros(rolled_parameters_red.shape)
    perturb = np.zeros(rolled_parameters_red.shape)
    
    grad = gradient(rolled_parameters_red, Y_red, R_red, num_features, 0)
    
    (rolled_parameters_red, Y_red, R_red, num_features)
    
    for i in range(n):
        perturb[i] = e
        loss1 = cost_function((rolled_parameters_red - perturb), Y_red, R_red, num_features, 0)
        loss2 = cost_function((rolled_parameters_red + perturb), Y_red, R_red, num_features, 0)
        numerical_gradient = (loss2 - loss1) / (2*e)
        to_check[i] = numerical_gradient
        perturb[i] = 0        
        
        print('---')
        print(to_check[i] - grad[i])

        if i == 20:
            break
        
    return (to_check - grad).sum()

In [16]:
check_gradient(rolled_parameters_red, Y_red, R_red, 3)

22.224856626532866
22.224350828203647
---
-8.993694677883468e-12
22.223846170594975
22.225361311210914
---
-1.0763834268345818e-11
22.224793705561407
22.224413747509132
---
1.3945733456921516e-11
22.224660545689616
22.224546906496514
---
2.2608581673466688e-12
22.224268474838134
22.224939004900705
---
6.038280986331301e-12
22.224656065876854
22.224551386185862
---
7.516098854409847e-12
22.224686966806374
22.224520485379756
---
8.021250330614293e-12
22.22411257657285
22.225094903165985
---
-7.72892860823049e-12
22.22468040390903
22.22452704815369
---
-3.8469227803261674e-13
22.224642084371528
22.224565367814602
---
-6.6756045136173725e-12
22.224377406171026
22.224830073567812
---
7.80397968469515e-12
22.224639060079646
22.22456839198307
---
-6.422806730910224e-12
22.22468410409921
22.22452334808692
---
-8.145040197859998e-12
22.22412946802695
22.225078011711886
---
-5.270450742500543e-12
22.22467776690244
22.22452968516028
---
-8.196443523900143e-12
22.225660543734868
22.22354693969398


-9.82658399095726e-12

In [17]:
# regularisation added in function via _lambda != 0

In [18]:
def collaborative_filtering(rolled_parameters, Y, R, num_features):
    result = op.minimize(fun=cost_function, x0=rolled_parameters, args=(Y, R, num_features), method='TNC', jac=gradient)
    return result.x    

In [19]:
res = collaborative_filtering(rolled_parameters_red, Y_red, R_red, 3)

22.224603725685675
22.224603346327743
22.22460334261128
22.22460144553869
320.1836909993188
11.646781877132122
2.8926462664989807
2.8926462137272146
2.8926463154176902
4.514584707835423
4.51458449409995
4.51458471630364
1.2512298550174321
1.2512298699575553
1.2512298538072686
1.2512298548796421
1.6818806148173175
1.6818807353849485
1.681880383304704
1.681880634935352
1.681880643643953
1.6818806051466844
1.6818805992829367
1.6280202108335817
1.628020222337687
1.6280202101509196
1.6740897536526511
1.674089755116052
1.6740897533478591
1.6740897532677597
1.6740897538427368
1.6804843493973305
1.6804841045512657
1.6804844751016708
1.6804843509457117
1.6804843355384629
1.6804843392314965
1.6804843576028494
1.679940767233778
1.6799407672025475
1.6799407671945839
1.6799407671771645
1.679940767176681
1.6797919750712742
1.679791975079109
1.6797919750409562
1.6798185467343119
1.6798185467381046
1.6798185467345792
1.6798615310175893
1.679861613792857
1.6798612635863248
1.6798615281545266
1.67986154

In [20]:
res = collaborative_filtering(rolled_parameters, Y, R, 10)

27918.64012454421
27918.64011520473
27918.640110371114
27918.640110401782
27918.640109977132
27918.64011265148
27918.640113202586
27918.640115545848
27918.640116586965
27918.640117896997
27918.64011818617
27918.640119385986
27918.640120043245
27918.640119990992
27918.640120531054
27918.640121311288
26671.603850347637
26671.603833773606
26671.60384255122
26671.603840587504
26671.603840237483
26671.603839963613
26671.60383684998
26671.603836929648
26671.60383812201
26671.60384034434
26671.60384251573
26671.603843053916
26671.603846800725
26354.42305354736
26354.42304904592
26354.42305106046
26354.423048918437
26354.42304887276
26354.423047555756
26354.423047203138
26354.423046892396
26354.423049010195
26354.423050079316
26354.42305092871
26253.93864761664
26253.93864581075
26253.93864641202
26253.938645199236
26253.938645394195
26253.93864446715
26253.938644700444
26253.938644868223
26253.93864532594
26253.938645924496
26209.21680332616
26209.216802691444
26209.2168030243
26209.216802515

In [21]:
res

array([ 1.08359062, -0.40205115,  1.25540469, ...,  1.28423703,
       -0.98349993, -0.53184838])

In [22]:
# predict 

In [35]:
number_movies, number_users = Y.shape
print(number_movies)
print(number_users)
X, theta = unroled_parameters(res, number_movies, number_users, 10) 

1682
943


In [30]:
Ypred = X @ theta.T

In [39]:
# rating for user 0
Ypred[:,0]

array([3.72616315, 3.13017623, 3.32097265, ..., 1.01872527, 1.71562154,
       2.2377962 ])